In [60]:
# Import Necessay Libraries.

from flask import Flask, render_template, jsonify, request
import datetime
from hashlib import sha256
import json
from pprint import pprint

In [61]:
# Class implementation for Block.

class Block:
    """
    Block representing a single block in a blockchain.

    Attributes:
        index (int): Block's unique index within the chain.
        data (str): Data stored in the block.
        timestamp (str): ISO 8601 timestamp of block creation.
        previous_hash (str): Hash of the previous block.
        nonce (int): Nonce used for Proof-of-Work (PoW).
        hash (str): Calculated SHA-256 hash of the block.
        is_valid (bool): Flag indicating block validity.
    """

    def __init__(self, index, data, timestamp, previous_hash):
        """Block constructor."""
        self.index = index  # Index of the block.
        self.data = data  # Data of the block.
        self.timestamp = (
            timestamp.isoformat()
            if isinstance(timestamp, datetime.datetime)
            else timestamp
        )  # Timestamp of the block.
        self.previous_hash = previous_hash  # Hash of the previous block.
        self.nonce = 0  # Nonce of the block
        self.hash = self.proof_of_work(Blockchain.difficulty) #  Proof-of-work.
        self.is_valid = True  # Initially every block is valid.

    def compute_hash(self):
        """Compute Hash of the block."""
        block_string = json.dumps(self.__dict__, sort_keys=True) # Block dictionary on which hash is computed.
        return sha256(block_string.encode()).hexdigest() # SHA-256 hash of the block.

    def proof_of_work(self, difficulty):
        """Proof-of-work of the block."""
        self.nonce = 0 # Nonce of the block.
        computed_hash = self.compute_hash() 
        while not computed_hash.startswith("0" * difficulty):
            self.nonce += 1
            computed_hash = self.compute_hash()
        return computed_hash

    def update_data(self, new_data, blockchain):
        self.data = new_data
        self.hash = self.compute_hash()
        self.is_valid = False
        # Mark all subsequent blocks as invalid
        current_block = self
        while current_block.index + 1 < len(blockchain.chain):
            next_block = blockchain.chain[current_block.index + 1]
            next_block.previous_hash = current_block.hash  # Update previous_hash to new hash
            next_block.hash = next_block.compute_hash()  # Recompute hash as it depends on previous_hash
            next_block.is_valid = False  # Each subsequent block also becomes invalid
            current_block = next_block

In [62]:
class Blockchain:
    """Blockchain implementation that ensures data integrity through PoW.

    Attributes:
        chain (list[Block]): The list of blocks forming the blockchain.
        difficulty (int): The difficulty level for Proof-of-Work (PoW).

    Methods:
        create_genesis_block(): Creates the initial block (genesis block)
                                in the blockchain.
        last_block (property): Returns the last block in the chain.
        add_block(data): Adds a new block containing the given data to the
                         end of the chain.
        is_block_valid(block): Validates the integrity of a block based on
                                PoW, previous hash, and internal validity flag.
        is_chain_valid(self): Validates the entire blockchain by checking
                              the validity of each block.
    """
    difficulty = 3

    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Genesis block creation."""
        genesis_block = Block(0, "Genesis Block", datetime.datetime.now(), "0") # Instantiate a new genesis block.
        genesis_block.hash = genesis_block.proof_of_work(Blockchain.difficulty) # Compute Hash of the genesis block.
        self.chain.append(genesis_block) # Add block to the chain.

    @property
    def last_block(self):
        """Return Last Block of the chain."""
        return self.chain[-1]

    def add_block(self, data):
        """Add new block to the chain."""
        new_block = Block(
            len(self.chain), data, datetime.datetime.now(), self.last_block.hash
        ) # Instantiate new block.
        new_block.hash = new_block.proof_of_work(Blockchain.difficulty) # Compute hash.
        new_block.is_valid = self.is_block_valid(new_block)  # Validity check of new block.
        self.chain.append(new_block) # Add block to chain.

    def is_block_valid(self, block):
        if block.index == 0:
            return block.hash.startswith("0" * self.difficulty) and block.previous_hash == "0"
        previous_block = self.chain[block.index - 1]
        valid_link = block.previous_hash == previous_block.hash
        valid_pow = block.hash.startswith("0" * self.difficulty)
        block.is_valid = valid_link and valid_pow and block.is_valid
        return block.is_valid

    def is_chain_valid(self):
        for block in self.chain:
            if not self.is_block_valid(block):
                return False
        return True

In [63]:
blockchain = Blockchain() # Instantiate the Blockchain.

In [64]:
blockchain.chain[0].__dict__  # Genesis Block.

{'index': 0,
 'data': 'Genesis Block',
 'timestamp': '2024-05-12T11:16:32.618331',
 'previous_hash': '0',
 'nonce': 5425,
 'hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'is_valid': True}

In [65]:
blockchain.add_block("BlockData1") # Add New Block.

In [66]:
for i in range(len(blockchain.chain)):
    pprint(blockchain.chain[i].__dict__, sort_dicts=False)
    print()

{'index': 0,
 'data': 'Genesis Block',
 'timestamp': '2024-05-12T11:16:32.618331',
 'previous_hash': '0',
 'nonce': 5425,
 'hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'is_valid': True}

{'index': 1,
 'data': 'BlockData1',
 'timestamp': '2024-05-12T11:16:32.733516',
 'previous_hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'nonce': 12046,
 'hash': '000af640ca33edc0214981dcf5fd2a454f5644dafa93f8bfa7e378e777169b89',
 'is_valid': True}



In [67]:
blockchain.add_block("BlockData2") # Add New Block.
blockchain.add_block("BlockData3") # Add New Block.

In [68]:
for i in range(len(blockchain.chain)):
    pprint(blockchain.chain[i].__dict__, sort_dicts=False)
    print()

{'index': 0,
 'data': 'Genesis Block',
 'timestamp': '2024-05-12T11:16:32.618331',
 'previous_hash': '0',
 'nonce': 5425,
 'hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'is_valid': True}

{'index': 1,
 'data': 'BlockData1',
 'timestamp': '2024-05-12T11:16:32.733516',
 'previous_hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'nonce': 12046,
 'hash': '000af640ca33edc0214981dcf5fd2a454f5644dafa93f8bfa7e378e777169b89',
 'is_valid': True}

{'index': 2,
 'data': 'BlockData2',
 'timestamp': '2024-05-12T11:16:32.909153',
 'previous_hash': '000af640ca33edc0214981dcf5fd2a454f5644dafa93f8bfa7e378e777169b89',
 'nonce': 1148,
 'hash': '000c015f15a1e340b766a44ec2cb8499f9cd4071f4082c79f33fad3fb546ef76',
 'is_valid': True}

{'index': 3,
 'data': 'BlockData3',
 'timestamp': '2024-05-12T11:16:32.923299',
 'previous_hash': '000c015f15a1e340b766a44ec2cb8499f9cd4071f4082c79f33fad3fb546ef76',
 'nonce': 4524,
 'hash': '000c0eb7de684ef7d49e68d89745a1

In [70]:
blockchain.chain[1].update_data("ManipulatedBlockData1", blockchain)
blockchain.is_chain_valid()

False

In [71]:
for i in range(len(blockchain.chain)):
    pprint(blockchain.chain[i].__dict__, sort_dicts=False)
    print()

{'index': 0,
 'data': 'Genesis Block',
 'timestamp': '2024-05-12T11:16:32.618331',
 'previous_hash': '0',
 'nonce': 5425,
 'hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'is_valid': True}

{'index': 1,
 'data': 'ManipulatedBlockData1',
 'timestamp': '2024-05-12T11:16:32.733516',
 'previous_hash': '0005704b2f4043b47fab6b484b356944b4b5a4b79d437f9dfc47746a774d75a0',
 'nonce': 12046,
 'hash': 'e81465c3513e1f9d4062c2a371c48e2efb0f328dd2a1d6497652adfdb07556fe',
 'is_valid': False}

{'index': 2,
 'data': 'BlockData2',
 'timestamp': '2024-05-12T11:16:32.909153',
 'previous_hash': 'e81465c3513e1f9d4062c2a371c48e2efb0f328dd2a1d6497652adfdb07556fe',
 'nonce': 1148,
 'hash': 'e60471c5c96a696ec552ff440753d8f7c85191ced30ffa66213c325db651f31f',
 'is_valid': False}

{'index': 3,
 'data': 'BlockData3',
 'timestamp': '2024-05-12T11:16:32.923299',
 'previous_hash': 'e60471c5c96a696ec552ff440753d8f7c85191ced30ffa66213c325db651f31f',
 'nonce': 4524,
 'hash': '26d82f4bc3c9259bf